In [15]:
import logging
import ipywidgets as wd
import polars as pl
from IPython.display import display, HTML




class App:
    price_data: pl.LazyFrame
    weather_data: pl.LazyFrame
    combined_data: pl.LazyFrame
    logger = logging.getLogger("App")
    logger.setLevel(logging.INFO)


    def read_price_data(self, path: str):
        """Read the price history."""
        schema = {
            "Datetime (Local)": pl.Datetime("ms"),
            "Price (EUR/MWhe)": pl.Float32,
        }
        price_data = pl.read_csv(path, schema=schema).lazy()
        price_data = price_data.rename({"Datetime (Local)": "time", "Price (EUR/MWhe)": "price"})
        self.price_data = price_data

    def read_weather_data(self, path: str):
        """Read weather data."""
        schema = {
            "time": pl.Datetime("ms"),
            "wsp_150.0": pl.Float32,
            "power": pl.Float32,
            "price": pl.Float32,
            "Income": pl.Float32
        }
        data = pl.read_csv(path, schema=schema).lazy().rename({"wsp_150.0": "wind_speed"})
        self.weather_data = data


    def _join_data(self):
        combined_data = self.weather_data.join(self.price_data, on="time", how="inner")
        self.combined_data = combined_data


app = App()
app.read_price_data("price_power_income.csv")

schema = {
    "time": pl.Datetime("ms"),
    "wsp_150.0": pl.Float32,
    "power": pl.Float32,
    "price": pl.Float32,
    "Income": pl.Float32
}
data = pl.read_csv("price_power_income.csv", schema=schema)

out = wd.Output(layout=wd.Layout(width="autopx", max_height="350px"))

with out:
    with pl.Config() as cfg:
        cfg.set_tbl_rows(-1)
        display(HTML(data._repr_html_()))


app = wd.Box((out,))


display(app)

Box(children=(Output(layout=Layout(max_height='350px', width='autopx')),))